### Retrival Augmentented Generation Day 2

In [1]:
%pip install langchain_cohere -q
%pip install spacy -q
%pip install psycopg2 -q
%pip install python-dotenv -q
#ignore error

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.34.51 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.2.1 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
transformers 4.31.0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.19.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [3]:
import boto3
import pandas as pd
import json
import time
import os
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
import dbconnection
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [4]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Cohere

In [5]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [6]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [7]:
# Let's generate a dense vector using Amazon Titan without using a np.array as a return value
def generate_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    #Note pgvector does not want a np.array as out manual method
    return(embedding)



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [8]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [9]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [10]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [11]:
def print_top_values(list_stuff: list, num_items: int) -> None:
    i=0
    for item in list_stuff:
        i=i+1
        if i>num_items:
            return None
        print(item)

In [12]:
# clean abstract text
#df = pd.read_csv('data/latest_research_articles.csv')
#df['abstract'] = df['abstract'].apply(clean_value)

#df
dft = pd.read_pickle('data/embedded_df.pkl')

### Advanced Retrieval Techniques
#### HyDE
A technique that optimizes semantic matching requires better semantic context.  What if we generated a document from the query that better match our stored document?

In [13]:
### Retrieval from embedded sources
#Now that we have a dataframe with embedded content of interest, we can use semantic similarity to retrieve the right data to feed to an LLM

# Given the following query let's generate context that more closely matches the embedded data
query = "What is the latest research for broken ribs in children"

#### Calling the LLM with Python
Before we embed the vector with the query let's transform the query into a fake article.  This article will likely have a larger semantic overlap than the original smaller question. Using Bedrock we will now call Anthropic Claude Sonnet to generate a fictitous article.


In [14]:
# Generate HyDE context

def generate_hyde_response(query_phrase):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    # Each model will named parameters which will likely be different depending on the providor
    model_kwargs =  { 
        "max_tokens": 400, # This is the maximum output tokens you want the model to use
        "temperature": 1,  # Temperature controls the randomness and creativity of the generated text.
        "top_k": 250,      # Top-k parameter determines the number of highest probability next word choices the model should conside
        "top_p": 0.9,      # Top-p sampling considers the cumulative probability distribution of the next word choices and sets a probability threshold
        "stop_sequences": ["\n\nHuman"],
    }
    # LangChain tooling
    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )
    
    human_prompt = "Given the following question \n {query} can you please generate a paragraph of text that answers the question. Be sure to use scientific \
                    medical terminology. Please just include the paragraph in your response."
    # Uses the messaging method which is required for all Claude 3 calls
    messages = [
        ("system", "You are a helpful assistant"),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)
        # LangChain at work
        chain = prompt | model | StrOutputParser()


        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"query": query_phrase})
        end_time = time.time()  # End timing
        print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno
        print(f"Errort: {exc_type}{exc_value}{exc_traceback} on {line_number}")

#### Differentiating the retrievals and ranking
 How can we evaluate if the cosine similarities are different enough?  Thresholds aren't consistent enough.  What about Z-Score?

In [18]:
def calculate_zscores(cosine_scores):
    zscores = []
    # Calculate the mean of the sample points
    mean = np.mean(cosine_scores)
    # Calculate the standard deviation of the sample points
    std_deviation = np.std(cosine_scores, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate the z-scores for each sample point
    z_scores = [(x - mean) / std_deviation for x in cosine_scores]

    return z_scores

Let's review our distribution of cosine scores and evaluate a threshold for selection.  We will review z-score and see how that compares.

In [19]:
# grab the cosine_scores from the results tuple
cosine_scores = [item[1] for item in results]
print("The top few cosine-scores")
print_top_values(cosine_scores,5)
z_scores = calculate_zscores(cosine_scores)
# Let's print the top 5 z-scores
print("The top few Z-scores")
print_top_values(z_scores, 5)



The top few cosine-scores
0.562133432897652
0.2578122448728413
0.2397991049472834
0.2397506375362562
0.2362506922746699
The top few Z-scores
11.251934861017451
4.800474308115775
4.4186045394746225
4.417577054032768
4.343379923892103


#### Sample values for cosine similarity I sampled for another project I used RAG on

<img src="zscore-dist.jpg" alt="Sample Z-Score Distributions" width="300" height="600">

In [20]:
# Using our new technique we will concat all relevant results and prepare to send that as context to our LLM
articles = ""
i=0
j=0
first_z_score = z_scores[0]
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    article_title = dft.iloc[item[0]]['title']
    abstract = dft.iloc[item[0]]['abstract']
    # If the highest Z score is 2x this record then likey not as significant
    if(first_z_score/2)<z_scores[i]:
        print(f"Abstract: '{article_title}' with a cosine match of: {item[1]} and Z-score of: {z_scores[i]}")
        articles = articles + " " + abstract
        j=j+1
    i=i+1
print(f"Grabbed the top {j} scores")    

Abstract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.562133432897652 and Z-score of: 11.251934861017451
Grabbed the top 1 scores


#### Generation
Now lets define a function that will call the LLM with the right prompt to generate a summary of the retrieved context

In [21]:
# Now let's take the records that are greater that 1/2 the top Z-score and send those to the LLM for an answer
def best_answer(data, question):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    model_kwargs =  { 
        "max_tokens": 2048,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"],
    }

    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    human_prompt = "You are to answer the question using the data in the following information.  Do not make up your answer, only use \
                    supporting data from the article, If you don't have enough data simply respond, I don't have enough information to answer that question. \
                    given the following article data {data} can you please give a concise answer to the following question. {question}"
    messages = [
        ("system", "You are a helpful assistant that can answer quesitons based on news articles you have been given."),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()

        # Chain Invoke
        
    
        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"data": data,"question": question})
        end_time = time.time()  # End timing
        #print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno

        return f"ERROR generating good answer: {exc_type}{exc_value}{exc_traceback} on {line_number}"


In [22]:
print("Here is the best answer I could find about ", query)
print(best_answer(articles, query))

Here is the best answer I could find about  What is the latest research for broken ribs in children
Based on the information provided in the article, the latest research focuses on improving the detection and localization of rib fractures in pediatric chest radiographs using computer vision and machine learning techniques. Specifically:

- The research implemented convolutional neural network (CNN) architectures like RetinaNet and YOLOv5, along with an "avalanche decision" scheme to dynamically adjust detection thresholds, to increase the sensitivity (recall) in detecting rib fractures.

- Multiple image preprocessing techniques and model ensembling were used to further enhance detection performance.

- Using a dataset of 1,109 pediatric chest radiographs manually labeled by radiologists, the best performing model achieved an F2 score of 0.725, approaching the expert inter-reader performance of 0.732.

- The goal was to develop methods that can identify all rib fractures to augment rad

#### RAG is rad
Let's explore the power of RAG with another example.  Using a larger document with chunking

In [23]:
# Load your large text document
with open("data/staff-report.txt", "r") as file:
    text = file.read()

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,  # Adjust the chunk size as needed
    chunk_overlap=512,  # Adjust the overlap between chunks as needed
)

# Split the text into chunks
chunks = text_splitter.split_text(text)

# create a dataframe with new chunk raw text
chunk_df = pd.DataFrame({'raw_text': chunks})

chunk_df['text_embed'] = chunk_df['raw_text'].apply(generate_titan_vector_embedding)


In [27]:
# Let's search our records for a good semantic search
#query = "What is the city replacing at the water treatment plant and why?"
#query = "What is the city recommending?"
query = "What will it cost to replace the mixer gearbox?"

query_vector = generate_titan_vector_embedding(query)
# This is a tuple of the article index and the cosine similarity score
results = []
# Iterate over each row in the DataFrame
for index, row in chunk_df.iterrows():
    # Extract the value from the specified column
    article_embedding = row['text_embed']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, cosine_similarity(article_embedding, query_vector))

results.sort(key=lambda x: x[1], reverse=True)
cosine_scores = [item[1] for item in results]
z_scores = calculate_zscores(cosine_scores)


print("The top few cosine-scores")
print_top_values(cosine_scores,5)
# Let's print the top 5 z-scores
print("The top few Z-scores")
print_top_values(z_scores, 5)

#save the articles that were a good match
articles = ""
i=0
j=0
first_z_score = z_scores[0]
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    chunk_txt = chunk_df.iloc[item[0]]['raw_text']
    # If the highest Z score is 2x this record then likey not as significant
    if(first_z_score/2)<z_scores[i]:
        print(f"Using chunk with a cosine match of: {item[1]} and Z-score of: {z_scores[i]}")
        articles = articles + " " + chunk_txt
        j=j+1
    i=i+1
print(f"Grabbed the top {j} scores")  
print("Here is the best answer I could find about ", query)
print(best_answer(articles, query))
  

The top few cosine-scores
0.5030754138386723
0.4098611106584495
0.3989779966941804
0.3701080283478417
0.3559540099772899
The top few Z-scores
2.047525838481718
1.2942966333060593
1.206354343319797
0.9730671484467585
0.8586939284197874
Using chunk with a cosine match of: 0.5030754138386723 and Z-score of: 2.047525838481718
Using chunk with a cosine match of: 0.4098611106584495 and Z-score of: 1.2942966333060593
Using chunk with a cosine match of: 0.3989779966941804 and Z-score of: 1.206354343319797
Grabbed the top 3 scores
Here is the best answer I could find about  What will it cost to replace the mixer gearbox?
According to the information provided, the total project cost to replace the Actiflo mixer gearboxes at the City's Water Treatment Plant is $500,322. The breakdown is:

Philadelphia Mixer Bearing Replacements: $416,935
Contingency (20%): $83,387
Total Project Cost: $500,322
